In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.manifold import TSNE
from ipywidgets import interactive, HBox, VBox

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

In [2]:
from plotly.offline import init_notebook_mode, iplot
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
primeiroCiC = ['116301', '118001', '118010', '113034', '140481', '145971']
segundoCiC = ['116319', '115045', '118028', '118036', '113042']
terceiroCiC = ['118044', '118052', '113956', '113051', '117366']


primeiroLic1 =['116793', '117366', '145971', '147389', '194221']
segundoLic1 = ['113034', '116301', '125156']
terceiroLic1 = ['115045', '116319', '125172']

primeiroLic2 =['113492', '194221', '113450', '113476']
segundoLic2 = ['113034', '116319', '191027', '150649', '194280']
terceiroLic2 = ['115045', '113093', '117366', '117889']

primeiroEngComp =['113034', '113093', '113476', '117528', '118001', '118010']
segundoEngComp = ['118028', '118036', '113042', '115045', '116319']
terceiroEngComp =['113051', '117242', '170054', '201600', '206075']


primeiroMec = ['114626', '114634', '118010', '113476', '113034', '118001', '168891']
segundoMec = ['168891', '113093', '118028', '118036', '115045', '168874']
terceiroMec = ['113051', '118044', '118052', '168769', '116319']

primeiroRedes = ['113034', '113093', '118001', '118010', '167959']
segundoRedes = ['113042', '115045', '118028', '118036', '169676']
terceiroRedes = ['113051', '113301', '167983', '169749']


undropableCol = ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]#, 'Ingresso_unb'

In [54]:
df0 = pd.read_csv("aluno_quant_Materias_Media_cotistas_0.csv", index_col=0)
df1 = pd.read_csv("aluno_quant_Materias_Media_cotistas_1.csv", index_col=0)
df2 = pd.read_csv("aluno_quant_Materias_Media_nao_cotistas_2.csv", index_col=0)
df3 = pd.read_csv("aluno_quant_Materias_Media_cotistas_3.csv", index_col=0)
df4 = pd.read_csv("aluno_quant_Materias_Media_cotistas_4.csv", index_col=0)
df5 = pd.read_csv("aluno_quant_Materias_Media_cotistas_5.csv", index_col=0)

In [55]:
dfComp = df0[undropableCol+primeiroLic1+segundoLic1+terceiroLic1].fillna(-1)
dfRedes = df1[undropableCol+primeiroRedes+segundoRedes+terceiroRedes].fillna(-1)
dfCic = df2[undropableCol+primeiroCiC+segundoCiC+terceiroCiC].fillna(-1) 
dfMec = df3[undropableCol+primeiroMec+segundoMec+terceiroMec].fillna(-1) 
dfEng = df4[undropableCol+primeiroEngComp+segundoEngComp+terceiroEngComp].fillna(-1) 

In [9]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [25]:
def formatColor(color):
    return 'rgb('+str(int(color[0]*255))+','+str(int(color[1]*255))+','+str(int(color[2]*255))+')'


    
def tsne(df, currentLabel = 'saiu'):
    a = df
    X = a.values
    X_embedded = TSNE(n_components=2).fit_transform(X)
    
    
    data = a
    Y_sklearn = X_embedded
    y = data[currentLabel].values
    numColors = len(data[currentLabel].unique())

    cmap = get_cmap(numColors+1)
    colorList = [cmap(i) for i in range(1, numColors+1)]


    traceArr = []
    for lab, col in zip(data[currentLabel].unique()[:1],
                        colorList):
        
        trace1 = go.Scatter(
            x=Y_sklearn[y==lab, 0],
            y=Y_sklearn[y==lab, 1],
            mode='markers',
            name = str(lab),
            marker=dict(
                size=5,
                color=formatColor(col),
                opacity=0.8
            )
        )
        traceArr.append(trace1)
    f = go.FigureWidget(traceArr)
    
    t = go.FigureWidget([go.Table(
    header=dict(values=["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[df[col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])
    
    def selection_fn(trace,points,selector):
        t.data[0].cells.values = [df.loc[points.point_inds][col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]]
    
    for el in f.data:
        el.on_selection(selection_fn)
    VBox((f,t))
    

In [24]:
tsne(dfCic)

In [82]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2))) 

In [106]:
df = dfCic
df = df.reset_index().drop(['index'], axis=1)
currentLabel = 'saiu'

a = df
X = a.values
X_embedded = TSNE(n_components=2).fit_transform(X)


data = a
Y_sklearn = X_embedded
y = data[currentLabel].values
numColors = len(data[currentLabel].unique())

cmap = get_cmap(numColors+1)
colorList = [cmap(i) for i in range(1, numColors+1)]


traceArr = []
mapPoints = []
for lab, col in zip(data[currentLabel].unique(),
                    colorList):
    mapPoints.append(data[y==lab].index)
    selectedPoint.append(data[y==lab].index)
    
    trace1 = go.Scatter(
        x=Y_sklearn[y==lab, 0],
        y=Y_sklearn[y==lab, 1],
        mode='markers',
        name = str(lab),
        marker=dict(
            size=5,
            color=formatColor(col),
            opacity=0.8
        ),

    )
    traceArr.append(trace1)
f = go.FigureWidget(data=traceArr, layout=go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    ))

t = go.FigureWidget([go.Table(
header=dict(values=["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"],
            fill = dict(color='#C2D4FF'),
            align = ['left'] * 5),
cells=dict(values=[df[col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]],
           fill = dict(color='#F5F8FF'),
           align = ['left'] * 5))])
def getSelector(index):
    def selection_fn(trace,points,selector):
        print(t.data[0].cells.values)
        if (points.trace_index == 0):
            t.data[0].cells.values = [df.loc[mapPoints[index][points.point_inds]][col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]]
        else:
            a = [df.loc[mapPoints[index][points.point_inds]][col] for col in ["curso","saiu","Cotista","Raça","Sexo","Tipo de Escola"]]
            b = t.data[0].cells.values
            t.data[0].cells.values = tuple([item1 + item2 for item1, item2 in zip(a, list(b))])
        print(t.data[0].cells.values)
    return selection_fn

for ind, el in enumerate(f.data):
    el.on_selection(getSelector(ind))
    
VBox((f,t))

    'data': [{'marker': {'color': 'rgb(7,255,0)', 'opacity': 0.8, 'size': 5},
  …

([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

ValueError: Length of passed values is 0, index implies 1

In [83]:
result = Diff(list(range(6)), [2])
result

[0, 1, 3, 4, 5]

In [90]:
print(f)

FigureWidget({
    'data': [{'marker': {'color': 'rgb(7,255,0)', 'opacity': 0.8, 'size': 5},
              'mode': 'markers',
              'name': '-1',
              'selectedpoints': [],
              'type': 'scatter',
              'uid': '21756594-11e8-45f5-810c-e8aae94cf800',
              'x': array([-54.846386, -42.665928, -56.67701 , ..., -34.611923, -39.756012,
                          -29.610842], dtype=float32),
              'y': array([24.116085 , 12.605512 , 12.523917 , ..., 13.889252 , -2.5531147,
                          15.7781   ], dtype=float32)},
             {'marker': {'color': 'rgb(0,15,255)', 'opacity': 0.8, 'size': 5},
              'mode': 'markers',
              'name': '1',
              'type': 'scatter',
              'uid': 'fcb55c84-ec67-421d-bc41-7cd10b8b8097',
              'x': array([ 48.838947  ,  61.545197  ,   0.22774959, ..., -39.692677  ,
                           34.73083   , -14.513406  ], dtype=float32),
              'y': array([ 14.59